# 1. Uczenie klasyfikatora

Trochę informacji: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [2]:
trdata_gen = ImageDataGenerator()
traindata = trdata_gen.flow_from_directory(directory="../data/Training",target_size=(224,224)) 

Found 67692 images belonging to 131 classes.


In [3]:
CLASSES = len(traindata.class_indices)

In [4]:
tsdata_gen = ImageDataGenerator()
testdata = tsdata_gen.flow_from_directory(directory="../data/Test", target_size=(224,224))

Found 22688 images belonging to 131 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top = False , input_shape=(224,224, 3))
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
##TO CHECK OUT- LEAVE ONLY LAST ON
flatten= Flatten()(base_model.layers[-1].output)
# add new classifier layers
class_dense = Dense(1024, activation='relu')(flatten) 
output = Dense(CLASSES, activation='softmax')(class_dense)
# define new model
## maybe add dropout and maybe one more dense
model_pretrained = Model(inputs=base_model.inputs, outputs=output)
# summarize
model_pretrained.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
##freeze all  base_model layers and train the last ones
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model_pretrained.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [9]:
EPOCHS = 50
BATCH_SIZE = 32
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 64
MODEL_FILE = 'models/vgg16-pretrained-rmsprop.model'

In [10]:
model_pretrained.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
checkpoint = ModelCheckpoint("vgg16_1_pretrained.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20, verbose=1, mode='auto')
#csv_logger = CSVLogger('./results/vgg16imagenetpretrained.csv')

In [13]:
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
history_pretrained = model_pretrained.fit(
    traindata,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=testdata,
    validation_steps=VALIDATION_STEPS,
    shuffle=True,
    callbacks=[checkpoint, early_stopper]
)


model.save(MODEL_FILE)

Epoch 1/50
12/32 [==========>...................] - ETA: 1:22 - loss: 410.6916 - acc: 0.0417

KeyboardInterrupt: 

1.2 SVM

In [14]:
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
flatten_svm= Flatten()(base_model.layers[-1].output)
# add new classifier layers
class_dense_svm = Dense(4096, activation='relu')(flatten_svm) ##maybe not relu...
model_svm = Model(inputs=base_model.input, outputs=class_dense_svm)
model_svm.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
model_svm.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
feature = model_svm.predict(traindata,steps=1000, verbose=1)


   4/1000 [..............................] - ETA: 1:10:03

In [ ]:
print(feature.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, traindata.labels[:32000])

print(X_train[0:4])

In [ ]:
from sklearn import svm
### NOT QUITE SURE OF IT
clf = svm.SVC(kernel='rbf') # choosing kernels

#Train the model using the training sets
clf.fit(X_train[0:1000], y_train[0:1000])

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)

print("Incredible accuracy of : ",acc)